# Efficient Yelp API Calls (Core)
Student Name: Nathalie Castro

In [29]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [27]:
!pip install simplejson

In [30]:
# Install tmdbsimple (only need to run once)
!pip install tqdm

In [31]:
# Load API Credentials
with open('/Users/castr/.secret/yelp_api.json') as f:
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [32]:
# set our API call parameters 
LOCATION = 'Chicago,IL'
TERM = 'Taco'

In [33]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_chi_taco.json"
JSON_FILE

'Data/results_in_progress_chi_taco.json'

In [34]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_chi_taco.json not found. Saving empty list to file.


In [35]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [36]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [37]:
## How many results total?
total_results = results['total']
total_results

7200

In [38]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [39]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

360

In [40]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [41]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [42]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/360 [00:00<?, ?it/s]

In [56]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f) 

In [57]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_chi_taco.json not found. Saving empty list to new file.
- 0 previous results found.


360

In [58]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/360 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [59]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,-HqkUZg3tu-ZAV1RhdzXHQ,tacotlan-chicago,Tacotlan,https://s3-media3.fl.yelpcdn.com/bphoto/7bAgkw...,False,https://www.yelp.com/biz/tacotlan-chicago?adju...,269,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,"{'latitude': 41.92469, 'longitude': -87.73466}","[delivery, pickup]",$$,"{'address1': '4312 W Fullerton Ave', 'address2...",+17736665259,(773) 666-5259,10300.582947
1,L12zP2yCgoKm_NLwW_HIDw,quesabirria-jalisco-chicago-9,Quesabirria Jalisco,https://s3-media3.fl.yelpcdn.com/bphoto/F7yhgt...,False,https://www.yelp.com/biz/quesabirria-jalisco-c...,15,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias...",5.0,"{'latitude': 41.85803128798182, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '1314 W 18th St', 'address2': '',...",+17737712123,(773) 771-2123,6613.300176
2,kdIJtpWmVhGDsgGyOyhzKg,birrieria-zaragoza-chicago,Birrieria Zaragoza,https://s3-media2.fl.yelpcdn.com/bphoto/bdWcX3...,False,https://www.yelp.com/biz/birrieria-zaragoza-ch...,615,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 41.80447, 'longitude': -87.72385}","[delivery, pickup]",$$,"{'address1': '4852 S Pulaski Rd', 'address2': ...",+17735233700,(773) 523-3700,3123.759159
3,YE4rhvPRtsr_NwahV48amg,tacos-el-pastor-53-chicago,Tacos El Pastor 53,https://s3-media2.fl.yelpcdn.com/bphoto/vXCILf...,False,https://www.yelp.com/biz/tacos-el-pastor-53-ch...,40,"[{'alias': 'tacos', 'title': 'Tacos'}]",4.0,"{'latitude': 41.79915, 'longitude': -87.58363}","[delivery, pickup]",$$,"{'address1': '5319 S Hyde Park Blvd', 'address...",+18722447498,(872) 244-7498,12773.368820
4,UBkrWseQzMo9qTPynYu-sQ,taqueria-los-altos-de-jalisco-cicero,Taqueria Los Altos De Jalisco,https://s3-media1.fl.yelpcdn.com/bphoto/r3yzb9...,False,https://www.yelp.com/biz/taqueria-los-altos-de...,28,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 41.8540458679199, 'longitude': -8...","[delivery, pickup]",$,"{'address1': '1922 S Cicero Ave', 'address2': ...",+17086565130,(708) 656-5130,2688.465721


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,kLp9aQ-1ihul0ygTFjJEeA,marias-kitchen-manteno,Maria's Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/WtIti1...,False,https://www.yelp.com/biz/marias-kitchen-manten...,14,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 41.2507248, 'longitude': -87.8315...",[delivery],$,"{'address1': '200 N Locust St', 'address2': ''...",+18154683208,(815) 468-3208,65168.267336
996,yu9_Y4Szsf3KAe_5qRpnBA,la-mera-mera-taqueria-libertyville-2,La Mera Mera Taqueria,https://s3-media4.fl.yelpcdn.com/bphoto/LxKNoe...,False,https://www.yelp.com/biz/la-mera-mera-taqueria...,27,"[{'alias': 'mexican', 'title': 'Mexican'}]",5.0,"{'latitude': 42.2600314, 'longitude': -87.9473...",[],NaN,"{'address1': '1428 S Milwaukee Ave', 'address2...",+12245135590,(224) 513-5590,50826.563909
997,5d_VUo7AMd5YF7XR0-_jvw,el-ranchito-chicago-6,El Ranchito,https://s3-media1.fl.yelpcdn.com/bphoto/mchmbp...,False,https://www.yelp.com/biz/el-ranchito-chicago-6...,65,"[{'alias': 'mexican', 'title': 'Mexican'}]",1.5,"{'latitude': 41.920255, 'longitude': -87.775396}","[pickup, delivery]",$$,"{'address1': '5959 W Grand Ave', 'address2': '...",+17737458010,(773) 745-8010,10470.052549
998,tL3lslj3Ji9YQvK9yVjLqg,bryannas-restaurant-and-bar-chicago-3,BryAnnas Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/S2juI8...,False,https://www.yelp.com/biz/bryannas-restaurant-a...,103,"[{'alias': 'salvadoran', 'title': 'Salvadoran'}]",4.0,"{'latitude': 41.985118, 'longitude': -87.697985}","[pickup, restaurant_reservation, delivery]",$$,"{'address1': '5695 N Lincoln Ave', 'address2':...",+18722085599,(872) 208-5599,17243.645744
999,8knHg1Bbalunt-fAzWym4A,taqueria-el-toreo-bensenville,Taqueria EL Toreo,https://s3-media3.fl.yelpcdn.com/bphoto/MST5kS...,False,https://www.yelp.com/biz/taqueria-el-toreo-ben...,9,"[{'alias': 'mexican', 'title': 'Mexican'}]",3.5,"{'latitude': 41.9311466, 'longitude': -87.9478...",[delivery],$,"{'address1': '217 W Grand Ave', 'address2': ''...",+16308600405,(630) 860-0405,21099.677658


In [61]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

3

In [62]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [63]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_chi_tacos.csv.gz', compression='gzip',index=False)